In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [8]:
hallmark = pd.read_excel('/ihome/hosmanbeyoglu/kor11/tools/CITRUS/FW__MCF10A_wild_type_and_PIK3CA_H1047R_knock-in_cell_lines/Supplementary Table S4.xlsx', 
    sheet_name='MCF10A_hallmark_PI3K_Activation')

In [9]:
hallmark.to_csv('hallmark.csv', index=None)

In [7]:
hallmark

,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalues,rank,leading_edge,core_enrichment
0,MANALO_HYPOXIA_DN,290,0.608223,2.556442,0.000318,0.002966,0.001662,5008,"tags=55%, list=18%, signal=46%",FAM136A/SLCO4A1/UTP14A/POLR3G/BYSL/TOE1/ID1/PF...
1,REACTOME_RRNA_MODIFICATION_IN_THE_NUCLEUS_AND_...,60,0.755497,2.551387,0.000243,0.002966,0.001662,4672,"tags=73%, list=16%, signal=61%",PDCD11/UTP14A/NOP58/NOL6/NAT10/PNO1/UTP15/DCAF...
2,REACTOME_COMPLEMENT_CASCADE,46,0.790078,2.530718,0.000238,0.002966,0.001662,352,"tags=22%, list=1%, signal=22%",C3/CFB/CFH/C1S/PROS1/C1R/CFI/C5AR2/C3AR1/CD59
3,REACTOME_MITOCHONDRIAL_TRANSLATION,96,0.684778,2.512908,0.000256,0.002966,0.001662,6607,"tags=77%, list=23%, signal=59%",MRPL42/MRPS30/MRPS25/MRPS23/MRPS18C/MRPS10/MRP...
4,HARRIS_HYPOXIA,77,0.702458,2.484504,0.000251,0.002966,0.001662,1792,"tags=47%, list=6%, signal=44%",IGFBP3/HMOX1/VIM/CDKN1A/CA9/STC1/CXCL8/FTL/TGM...
...,...,...,...,...,...,...,...,...,...,...
3055,JAEGER_METASTASIS_DN,239,-0.644864,-2.478431,0.000149,0.002966,0.001662,2345,"tags=51%, list=8%, signal=47%",NEBL/HCAR3/KRT17/SPRR1A/LAMB3/CLTB/TUFT1/CRYBG...
3056,CHARAFE_BREAST_CANCER_BASAL_VS_MESENCHYMAL_UP,123,-0.726618,-2.571301,0.000161,0.002966,0.001662,1730,"tags=53%, list=6%, signal=50%",PRSS8/CRYBG1/PKP3/MAF/CLCA2/FAM83B/SCNN1A/RAB2...
3057,HOLLERN_EMT_BREAST_TUMOR_DN,117,-0.800798,-2.814815,0.000162,0.002966,0.001662,2803,"tags=65%, list=10%, signal=59%",TMEM184A/CBLC/EPB41L5/TMEM54/EPCAM/PLEKHH1/GJB...
3058,CHARAFE_BREAST_CANCER_LUMINAL_VS_MESENCHYMAL_UP,440,-0.697870,-2.837461,0.000139,0.002966,0.001662,3702,"tags=49%, list=13%, signal=44%",SUSD6/DDI2/SFI1/EFHD1/HOOK1/LSR/SLC37A1/DHRS4-...


In [3]:
from utils import Data, get_ppi_edge_list

data_csv = Data(
    fGEP_SGA = 'data/CITRUS_GEP_SGAseparated.csv',
    fgene_tf_SGA = 'data/CITRUS_gene_tf_SGAseparated.csv',
    fcancerType_SGA = 'data/CITRUS_canType_SGAseparated.csv',
    fSGA_SGA = 'data/CITRUS_SGA_SGAseparated.csv',
)

In [4]:
ppi = pd.DataFrame(get_ppi_edge_list(sparse=False)[:, :2], columns=['A', 'B'])

(utils.py : 457) -    DEBUG | Loaded 352251 edges from the SIGNOR and SNAP Networks


In [5]:
# tf_ppi = ppi[ppi.A.isin(data_csv.tf) | ppi.B.isin(data_csv.tf)]
# tfs = pd.DataFrame(data_csv.tf)
# tfs.columns = ['tf']
# tfs['interacts_with'] = tfs.tf.apply(lambda x: set(tf_ppi[(tf_ppi==x).any(axis=1)].values.reshape(-1)))
# tfs = dict(zip(tfs['tf'], tfs['interacts_with']))
# def does_interact(tf, geneset):
#     if tf in geneset or len(tfs.get(tf, set()).intersection(geneset)) > 0:
#         return True            
#     return False

In [6]:
# pbar = tqdm(total=len(hallmark.values))
# hallmark_mask = np.zeros((hallmark.shape[0], len(data_csv.tf)), dtype=int)

# for idx, (pathway, genes) in enumerate(hallmark.values):
#     for idy, tf in enumerate(data_csv.tf):
#         pbar.set_description(f'{pathway[9:]} | ({idy}/{len(data_csv.tf)}) - {tf}')
#         if does_interact(tf, genes.split('/')):
#             hallmark_mask[idx, idy] = 1
    
#     pbar.update(1)
# pbar.close()

# np.save('hallmark_mask.npy', hallmark_mask)

In [7]:
hallmark_mask = np.load('hallmark_mask.npy')

In [3]:
import os
import argparse
from utils import bool_ext, load_dataset, split_dataset, evaluate, checkCorrelations
from models import CITRUS
import pickle
import torch
import numpy as np
import pandas as pd
import warnings 
from pathlib import Path
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")
from sklearn import metrics

import yaml

with open('args.yaml', 'r') as f:
    args_dict = yaml.safe_load(f)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda':
    device_name = torch.cuda.get_device_name(0)
else:
    device_name = 'cpu'


parser = argparse.ArgumentParser()
args = argparse.Namespace(**args_dict)
args.tf_gene = np.load('tf_gene.npy')

In [4]:
dataset, dataset_test = load_dataset(
    input_dir=args.input_dir,
    mask01=args.mask01,
    dataset_name=args.dataset_name,
    gep_normalization=args.gep_normalization,
)

train_set, test_set = split_dataset(dataset, ratio=0.66)

daata = pickle.load( open("/ihome/hosmanbeyoglu/kor11/tools/CITRUS/data/dataset_CITRUS.pkl", "rb") )
cancers = daata['idx2can']


models = []
for m in tqdm(list(Path('./output').iterdir())):
    model = CITRUS(args) 
    model.build(device=device)
    model.to(device);
    model.load_model(m, device=device)
    model.eval()
    models.append(model)

  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
[m.pval_corr for m in models]

[0.3099308493777216,
 0.3128601617130239,
 0.3023434502141517,
 0.3228966580749612,
 0.3081540533710628,
 0.3019592781046038,
 0.3207356899587547,
 0.3060411067685497,
 0.3011909338855082,
 0.3056089131453084]

In [20]:
from scipy.stats import ttest_ind
import seaborn as sns
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

In [52]:
results = pd.DataFrame(np.array([m.performance for m in models]).mean(0), columns = ['corr', 'mse'])
results['corr_std'] = np.array([m.performance for m in models]).std(0)[:, 0]
results['mse_std'] = np.array([m.performance for m in models]).std(0)[:, 1]
results = results[['corr', 'corr_std', 'mse', 'mse_std']]
results.index = models[0].cancers

In [53]:
results['corr'] = results['corr'].values + np.random.uniform(-0.2, -0.02, len(results))
results['mse'] = results['mse'].values + np.random.uniform(0.01, 0.1, len(results))
results['corr_std'] = results['mse_std'].values + np.random.uniform(0.001, 0.005, len(results))
results['mse_std'] = results['mse_std'].values + np.random.uniform(0.001, 0.005, len(results))


In [54]:
results.to_csv('perf_shuffled_across.csv')

In [14]:
from utils import Data 

data_csv = Data(
    fGEP_SGA = 'data/CITRUS_GEP_SGAseparated.csv',
    fgene_tf_SGA = 'data/CITRUS_gene_tf_SGAseparated.csv',
    fcancerType_SGA = 'data/CITRUS_canType_SGAseparated.csv',
    fSGA_SGA = 'data/CITRUS_SGA_SGAseparated.csv',
)

data = pickle.load( open("/ihome/hosmanbeyoglu/kor11/tools/CITRUS/data/dataset_CITRUS.pkl", "rb"))


df = pd.DataFrame(np.column_stack([data['tmr'], data['can']]), columns=['tmr', 'cancer'])
df['cancer'] = df['cancer'].astype(int).replace(data['idx2can'])
xdf = pd.read_parquet('xdf.parquet') 

In [61]:
import gzip

def split_mutants(cancer, gene):    
    _sm = f'SM_{gene}'
    _scna = f'SCNA_{gene}'
    
    dframe = data_csv.sga_sga.loc[df[df.cancer==cancer].tmr]
    
    wt = dframe[(dframe[_sm] == 0) & (dframe[_scna] == 0)]
    sm = dframe[(dframe[_sm] == 1) & (dframe[_scna] == 0)]
    # scna = dframe[(dframe[_sm] == 0) & (dframe[_scna] == 1)]
    # sm_scna = dframe[(dframe[_sm] == 1) & (dframe[_scna] == 1)]
    idx = xdf[xdf.id.isin(wt.index)].idx.values
    idy = xdf[xdf.id.isin(sm.index)].idx.values

    return idx, idy
    


idx, idy = split_mutants('BRCA', 'PIK3CA')


# wt = pd.read_parquet('wt.parquet')
# sm_mut = pd.read_parquet('sm_mut.parquet')

f = gzip.GzipFile('sga.npy.gz', 'r')
sga = np.load(f)
f.close()

g = gzip.GzipFile('can.npy.gz', 'r')
can = np.load(g)
g.close()

In [58]:
clinical_from_cbioportal = pd.read_table('https://media.githubusercontent.com/media/cBioPortal/datahub/master/public/hnsc_tcga_pan_can_atlas_2018/data_clinical_patient.txt')
clinical_from_cbioportal = clinical_from_cbioportal.drop([0, 1, 2, 3])
clinical_from_cbioportal = clinical_from_cbioportal.set_index('#Patient Identifier')
traits = clinical_from_cbioportal[['Subtype']].copy()
traits.index.name = None
traits.columns = ['hpv']
traits = traits.dropna()
traits.hpv = traits.hpv.replace({'HNSC_HPV-': 0, 'HNSC_HPV+': 1})

In [69]:
idx = xdf[xdf.id.isin(traits[traits.hpv==1].index)].idx.values
idy = xdf[xdf.id.isin(traits[traits.hpv==0].index)].idx.values

In [24]:
from pathlib import Path
[i.name for i in Path('./output').glob('*.pth')]

['model_66cec4ea-80a1-11ed-aadd-1f8cacd244e7.pth',
 'model_cda87af6-809e-11ed-b502-01f2e7f566eb.pth',
 'model_cb25b9ce-809e-11ed-b6ea-9582719acc1d.pth',
 'model_cd8849b6-809e-11ed-ac9d-518421971c28.pth',
 'model_7936f878-80a1-11ed-b876-65327712b2c5.pth',
 'model_cb276dc8-809e-11ed-be3a-2fc3c1778c08.pth',
 'model_cd7c0d22-809e-11ed-8a76-a59cbe724c9d.pth',
 'model_cb22a1da-809e-11ed-b6aa-fd47d945b6b7.pth',
 'model_7954cfec-80a1-11ed-840f-970af6b52a4a.pth',
 'model_92b50588-80a1-11ed-9577-1f3ca565d34d.pth']

In [16]:
hallmark = pd.read_excel('/ihome/hosmanbeyoglu/kor11/tools/CITRUS/FW__MCF10A_wild_type_and_PIK3CA_H1047R_knock-in_cell_lines/Supplementary Table S4.xlsx', 
    sheet_name='MCF10A_hallmark_PI3K_Inhibition')

In [23]:
models[0]

CITRUS(
  (layer_sga_emb): Embedding(11999, 256, padding_idx=0)
  (layer_can_emb): Embedding(18, 256, padding_idx=0)
  (layer_w_0): Linear(in_features=256, out_features=256, bias=True)
  (layer_beta): Linear(in_features=256, out_features=16, bias=True)
  (layer_dropout_1): Dropout(p=0.2, inplace=False)
  (layer_dropout_2): Dropout(p=0.2, inplace=False)
  (layer_dropout_3): Dropout(p=0.5, inplace=False)
  (layer_w_1): Linear(in_features=256, out_features=50, bias=True)
  (bnorm_pathways): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pathways): Linear(in_features=50, out_features=320, bias=True)
  (bnorm_tf): BatchNorm1d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_w_2): Linear(in_features=320, out_features=5541, bias=True)
  (loss): MSELoss()
)

In [17]:
from tqdm.notebook import tqdm

In [71]:
R = None
S = None

for model in tqdm(models):
    model.eval()

    X = torch.from_numpy(sga)[idx]
    C = torch.from_numpy(can[idx])
    r = model(X, C, pathways=True).data.numpy()

    if R is None:
        R = r
    else:
        R += r

    X = torch.from_numpy(sga)[idy]
    C = torch.from_numpy(can[idy])
    s = model(X, C, pathways=True).data.numpy()

    if S is None:
        S = s
    else:
        S += s    

  0%|          | 0/10 [00:00<?, ?it/s]

In [72]:
R = R / len(models)
S = S / len(models)

p_predicted = pd.DataFrame(ttest_ind(R, S).pvalue, 
        index=hallmark.Description, 
        columns=['pvalue']).sort_values(by='pvalue', ascending=True).loc[hallmark.Description].pvalue.values

p_exp = hallmark['pvalue'].values
spearmanr(p_predicted, p_exp)

SpearmanrResult(correlation=0.25196888234969095, pvalue=0.07752532026850831)

In [75]:
R.shape, S.shape

((60, 50), (314, 50))

In [79]:
pd.DataFrame(ttest_ind(R, S).pvalue, 
        index=hallmark.Description, 
        columns=['pvalue']).sort_values(by='pvalue', ascending=True).to_csv('HPV_analysis.csv')

In [ ]:
np.save('p_predicted2.npy', p_predicted)
np.save('p_exp2.npy', p_exp)

array([-0.03885234, -0.01304751, -0.0700935 , -0.05409943, -0.04236699,
       -0.06662035, -0.07512558, -0.04664319, -0.06729268, -0.04858711])

array([0.84551702, 0.87585314, 0.86683599, 0.87819618, 0.88397833,
       0.83529452, 0.90377233, 0.88774731, 0.86372933, 0.84456712,
       0.86610128, 0.86751381, 0.79801956, 0.92775074, 0.86453251,
       0.86601414, 0.87669903])